In [ ]:
# First initialize GsSession; external users should substitute client_id and client_secret fields with their credentials
from gs_quant.session import GsSession, Environment
GsSession.use(Environment.PROD, client_id=None, client_secret=None)

# Discover your Macro risk with Quant Insight models

Quant Insight's Macro risk models provide an insight in the relationship between movement in an asset price and
macroeconomic factors such as economic growth, monetary policy and commodity prices. The goal is to understand how much
of the movement in the asset price is attributable to those macroeconomic factors. The GS quant class `MacroRiskModel`
provides an array of functions that query macro risk model data. In this tutorial, we will look at querying all the
available macro risk model data.

Currently, the macro risk models that are available for programmatic access are below:

| Risk Model Name             | Risk Model Id |
|-----------------------------|---------------|
| US Equity Model (Long Term) | QI_US_EQUITY_LT |
| EU Equity Model (Long Term) | QI_EU_EQUITY_LT |
| UK Equity Model (Long Term) | QI_UK_EQUITY_LT |
| APAC Equity Model (Long Term) | QI_APAC_EQUITY_LT |

## Macro Factor Data

Macro Factors are grouped within a factor category.
Here are some examples of macro factor categories along with a list of macro factors in that category:

| Macro Factor Category | Macro Factors                                              |
|-----------------------|------------------------------------------------------------|
| Inflation             | US 5Y Infl. Expec., US 2Y Infl. Expec., US 10Y Infl. Expec.|
| Economic Growth       | Japan GDP, Euro GDP, China GDP                             |
| CB Rate Expectations  | Fed Rate Expectations                                      |
| Energy                | WTI                                                        |
| Risk Aversion         | VDAX, VIX, VXEEM, Gold Silver Ratio                        |

For more macro factor categories and their descriptions, see the factor glossary.

#### Get All Available Macro Factor Categories

We can leverage `get_factor_data` in the `MacroRiskModel` class to get all the available macro factor categories
in the model.


In [ ]:
from gs_quant.models.risk_model import MacroRiskModel, FactorType
import datetime as dt

start_date = dt.date(2022, 4, 1)

model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

# Get the factors in dataframe with name, and type
factor_category_data = model.get_factor_data(start_date=start_date, factor_type=FactorType.Factor).set_index('identifier').sort_values(by=["factorCategory"])
factor_category_data = factor_category_data.set_index("factorCategoryId").drop(columns={"name", "type"}).drop_duplicates()
factor_category_data = factor_category_data.rename_axis("Factor Category Id").rename(columns={"factorCategory": "Factor Category"})
display(factor_category_data)


#### Get All Available Macro Factors

Within each macro factor category, we have several macro factors that are grouped together.

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel, FactorType
import datetime as dt
import pandas as pd

start_date = dt.date(2022, 4, 1)

model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

# Get the factors in dataframe with name, and type
factor_data = model.get_factor_data(start_date=start_date, factor_type=FactorType.Factor).set_index('identifier').rename(columns={"name": "Factor", "factorCategory": "Factor Category"}).sort_values(by=["Factor Category"]).drop(columns={"type", "factorCategoryId"})
factor_data = factor_data.set_index("Factor Category").stack().to_frame().droplevel(level=1).rename(columns={0: "Factor"})
display(factor_data)

Further, for a more granular view, we can get all the macro factors that are grouped within a factor category.
Below is a list of all macro factors in the factorCategory "Risk Aversion".

In [ ]:
macro_factors_in_risk_aversion = factor_data.groupby("Factor Category").get_group("Risk Aversion")
display(macro_factors_in_risk_aversion)

## Asset Data

The metrics below are instrumental in providing insight in the relationship between movement of asset price and macro
factors. These key metrics are outlined below:

| Metric                    | Description    |
|----------------------------|---------------|
| `Universe Macro Factor Sensitivity` | Percentage change in asset price for 1 standard deviation move up in the macro factor. |
| `R Squared (Model Confidence)`               | Gauge of how sensitive the asset is to macroeconomic forces.  Typically values above 65% are considered to show strong explanatory power or ‘confidence’. |
| `Fair Value Gap`          | The difference between the actual price of an asset and the Qi Model Value.  This is quoted both in absolute (in percentage) and in standard deviation terms.  |
| `Specific Risk`            | Annualized idiosyncratic risk or error term which is not attributable to macro factors in percent units. |


## Get  your Portfolio Exposure to Macro Factors and Categories

Given a portfolio, we can get its cash exposure to a 1 standard deviation move up in each macro factor. The exposures are
calculated by multiplying each asset notional amount to their sensitivity to each factor, which gives us individual asset
 exposure to each macro factor. We then aggregate per asset exposures to get portfolio cash exposure to each macro \
 factor.
We can leverage the function `get_macro_exposure_table` of the `PortfolioManager` class. Note that exposure is expressed
in the currency of the notional value of the portfolio.

We can get portfolio exposure to both macro factor categories and macro factors. We can thus gain an insight
in which factor categories and macro factors within these categories are driving the portfolio.

#### Exposure to Macro Factor Categories

We need to pass the following parameters to the function:
* `macro_risk_model`: The model to base your exposure data on
* `date`: date for which to get exposure
* `group_by_factor_category`: whether to get exposure per factor category.

The result will be sorted from top positive drivers to top negative drivers

Note that, in the result, the macro factor categories are ranked from most positive exposure to most negative portfolio exposure

Below:
* We get asset level exposure to each factor category.
* The last row of the resulting dataframe is the portfolio exposure to each factor category (asset level exposure aggregated).

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel
from gs_quant.markets.portfolio_manager import PortfolioManager
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np
import pandas as pd

# For Styling
def style_negative(v, props=''):
    return props if v < 0 else None
styles = [{'selector': "caption", 'props': [("font-size", "200%"), ("font-weight", "bold"), ("color", 'black')]},
          {'selector': 'thead', 'props': [("background-color", "silver"), ("font-size", "12px")]},
          {'selector': 'tr', 'props': [("background-color", "whitesmoke"), ("font-size", "12px")]},
          {'selector': 'td', 'props': [("background-color", "aliceblue"), ("font-size", "12px")]}]

# Get the model
model = MacroRiskModel.get("QI_US_EQUITY_LT")

# Input the portfolio id
portfolio_id = "MPE2KR4BC4RKQ5KE"
pm = PortfolioManager(portfolio_id)
date = dt.date(2022, 5, 2)

category_index_label = "Portfolio Exposure Per Macro Factor Category"
factor_index_label = "Portfolio Exposure Per Macro Factor"

# Get raw exposure table
exp_factor_category_df = pm.get_macro_exposure_table(macro_risk_model=model, date=date, group_by_factor_category=True).rename_axis(["Factor Category", "Factor"], axis='columns')
asset_info = exp_factor_category_df["Asset Information"]
exp_factor_category_df = exp_factor_category_df.drop([factor_index_label, category_index_label], axis='index')
factor_categories = set([f[0] for f in exp_factor_category_df.columns.tolist()])

# Build factor category exposure table
exp_factor_category_df = pd.concat([exp_factor_category_df[factor_category].agg(np.sum, axis='columns').to_frame().rename(columns={0: factor_category}) for factor_category in factor_categories if factor_category != 'Asset Information'], axis='columns')
exp_factor_category_df = pd.concat([exp_factor_category_df, exp_factor_category_df.agg(np.sum).to_frame().rename(columns={0: "Total Exposure"}).transpose()])
exp_factor_category_df = pd.concat([asset_info, exp_factor_category_df], axis='columns')
exp_factor_category_df.loc["Total Exposure", "Notional"] = exp_factor_category_df.loc[category_index_label, "Notional"]
exp_factor_category_df.loc["Total Exposure", "Asset Name"] = "Total"
exp_factor_category_df = exp_factor_category_df.drop([factor_index_label, category_index_label], axis='index').set_index("Asset Name", drop=True).rename_axis("Asset Name").sort_values(by=["Total"], axis='columns', ascending=False).rename_axis("Factor Category", axis='columns')

# Limit how many rows will be displayed
if exp_factor_category_df.shape[0] > 20:
    exp_factor_category_df = pd.concat([exp_factor_category_df.head(10), exp_factor_category_df.tail(10)])

# Per asset exposure
exposure_styler = exp_factor_category_df.style.set_caption("Portfolio Cash Exposure to Factor Categories").format('${:,.0f}').applymap(style_negative, props='color:red;').set_table_styles(styles)
display(exposure_styler)

# Portfolio total exposure
total_exposure = exp_factor_category_df.loc["Total", :].drop(index="Notional").to_frame()
styles.pop(0)
display(total_exposure.style.format('${:,.0f}').applymap(style_negative, props='color:red;').set_table_styles(styles))

# Plot
fig, ax = plt.subplots(figsize=(25, 10))
sns.barplot(data=total_exposure.reset_index(), x="Factor Category", y='Total', ax=ax)
ax.set_xlabel("Factor Category", fontsize="x-large")
ax.set_ylabel("Portfolio Notional Change for 1 std Move in the Factor Category", fontsize="x-large")
ax.set_title("Portfolio Exposure to Factor Categories", fontsize="xx-large")
ax.tick_params(axis='x', labelrotation = 50)
plt.show()

#### Exposure to Macro Factors
Once we get portfolio exposure to factor categories, we can get a granular view of the portfolio exposure to individual macro factors within that factor category.

Here are the user inputs:
* `model`: The model we are using.
* `factor_categories`: A list of factor categories whose factors we want exposure for. If empty, it will default to returning exposure to the top 2 positive and negative macro drivers
* `portfolio_id`: The portfolio we are getting macro exposure for.
* `date`: Date 

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel
from gs_quant.markets.portfolio_manager import PortfolioManager
from IPython.display import display
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import seaborn as sns

# For Styling
def style_negative(v, props=''):
    return props if v < 0 else None
styles = [{'selector': "caption", 'props': [("font-size", "200%"), ("font-weight", "bold"), ("color", 'black')]},
          {'selector': 'thead', 'props': [("background-color", "silver"), ("font-size", "12px")]},
          {'selector': 'tr', 'props': [("background-color", "whitesmoke"), ("font-size", "12px")]},
          {'selector': 'td', 'props': [("background-color", "aliceblue"), ("font-size", "12px")]}]

# model id
model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

# date
date = dt.date(2022, 5, 2)

# Add Factor categories whose factors you want exposure for
factor_categories = ["Real Rates", "Inflation", "Corporate Credit"]

portfolio_id = "MPE2KR4BC4RKQ5KE"
pm = PortfolioManager(portfolio_id)

category_index_label = "Portfolio Exposure Per Macro Factor Category"
factor_index_label = "Portfolio Exposure Per Macro Factor"

# Get raw exposure table
exp_factor_df = pm.get_macro_exposure_table(macro_risk_model=model, date=date, group_by_factor_category=True).rename_axis(["Factor Category", "Factor"], axis='columns').drop(index=category_index_label)
asset_info = exp_factor_df[["Asset Information"]]
exp_factor_df = exp_factor_df.drop([("Asset Information", "Asset Name"), ("Asset Information", "Notional")],
                                   axis='columns')
if factor_categories:
    exp_factor_df = exp_factor_df[factor_categories]
else:
    drivers = list(dict.fromkeys(exp_factor_df.columns.get_level_values(0).tolist()))
    top_drivers = drivers[0:2] + drivers[-2:]
    exp_factor_df = exp_factor_df[top_drivers]

exp_factor_df = pd.concat([asset_info, exp_factor_df], axis='columns')
exp_factor_df.loc[factor_index_label, ("Asset Information", "Asset Name")] = "Total"
exp_factor_df = exp_factor_df.set_index(("Asset Information", "Asset Name"), drop=True).rename_axis("Asset Name").rename_axis(["Factor Category", "Factor"], axis='columns')

# Stylers
if exp_factor_df.shape[0] > 20:
    exp_factor_df = pd.concat([exp_factor_df.head(10), exp_factor_df.tail(10)])

exposure_styler = exp_factor_df.style.set_caption("Portfolio Cash Exposure to Macro Factors").format('${:,.0f}').applymap(style_negative, props='color:red;').set_table_styles(styles)
display(exposure_styler)

total_exposure = exp_factor_df.loc["Total", :].drop(index=("Asset Information", "Notional")).to_frame()
styles.pop(0)
display(total_exposure.style.format('${:,.0f}').applymap(style_negative, props='color:red;').set_table_styles(styles))

# Plot the portfolio percent change if every  factor category shifted 1 std
fig, ax = plt.subplots(figsize=(30, 10))
sns.barplot(data=total_exposure.sort_values(by=["Total"], ascending=False).reset_index(), x="Factor", y='Total', ax=ax)
ax.set_xlabel("Macro Factor", fontsize="x-large")
ax.set_ylabel("Portfolio Notional Change for 1 std Move in the Macro Factor", fontsize="x-large")
ax.set_title("Portfolio Exposure to Macro Factors", fontsize="xx-large")
ax.tick_params(axis='x', labelrotation=60)
plt.show()


### Run empirical stress tests to observe impact on portfolio

We can bump up or down any combination of macro factors and see the impact on the portfolio. Since your portfolio's
sensitivities to each macro factor are independent of each other, we can isolate a (or a combination of) macro factor categories and factors,
and assign a shift up or down in standard deviation and observe the change in the portfolio notional value.

Below, we define a class `MacroScenario` that takes in a date and a dictionary of macro factor categories or macro factors and their suggested standard
deviation shifts.

In [ ]:
from gs_quant.session import GsSession, Environment
from gs_quant.models.risk_model import MacroRiskModel
from gs_quant.markets.factor import Factor
from gs_quant.markets.portfolio_manager import PortfolioManager
from typing import Dict
import datetime as dt
import pandas as pd
import numpy as np

GsSession.use(Environment.PROD, client_id=None, client_secret=None)

class MacroScenario:
    def __init__(self, date: dt.date, factor_std_shifts: Dict[Factor, float]):
        self.date = date
        self.factor_std_shifts = factor_std_shifts

def macro_stress_test(portfolio_exposure_table: pd.DataFrame,
                      scenario: MacroScenario,
                      is_factor_category_stress: bool) ->  pd.DataFrame:

    """
    For a given portfolio, find how shifts in standard deviation in the selected macro factors affects
    the portfolio's notional value.
    :param portfolio_exposure_table: exposure of the portfolio to the requested macro factors
    :param scenario: scenario to stress for. Contains date and a dict of factors and respective shifts in standard
    deviation
    :return: portfolio notional change and change in portfolio exposure for each macro factor
    """

    macro_factor_std_shifts = scenario.factor_std_shifts
    factor_dict = {factor.name: std_shift for factor, std_shift in macro_factor_std_shifts.items()}
    if is_factor_category_stress:
        row_name = "Portfolio Exposure Per Macro Factor Category"
        old_portfolio_notional = portfolio_exposure_table.loc[row_name, ("Asset Information", "Notional")]
        factor_cols = set(portfolio_exposure_table.columns.get_level_values(0).values.tolist())
        factor_cols.remove("Asset Information")
        portfolio_exposure_table.columns = portfolio_exposure_table.columns.droplevel(level=1)
        portfolio_exposure_table = portfolio_exposure_table.loc[:, ~portfolio_exposure_table.columns.duplicated(keep='first')]
    else:
        row_name = "Portfolio Exposure Per Macro Factor"
        old_portfolio_notional = portfolio_exposure_table.loc[row_name, "Notional"]
        factor_cols = portfolio_exposure_table.columns.tolist()
        factor_cols.remove("Asset Name")
        factor_cols.remove("Notional")

    portfolio_change_df = pd.DataFrame(np.zeros((3, len(factor_cols))),
                                       index=["Standard Deviation Shift", "Portfolio Notional Change",
                                              "Percent Change"],
                                       columns=factor_cols)

    for factor, std_shift in factor_dict.items():
        old_exposure = portfolio_exposure_table.loc[row_name, factor]
        new_exposure = old_exposure * std_shift
        portfolio_change_df.loc["Standard Deviation Shift", factor] = std_shift
        portfolio_change_df.loc["Portfolio Notional Change", factor] = new_exposure
        portfolio_change_df.loc["Percent Change", factor] = (new_exposure / old_portfolio_notional) * 100

    total_sum = portfolio_change_df.agg(np.sum, axis="columns").to_frame()
    portfolio_change_df = pd.concat([portfolio_change_df, total_sum], axis='columns').rename(columns={0: "Total"})
    portfolio_change_df.loc["Standard Deviation Shift", "Total"] = np.NAN

    return portfolio_change_df

#### Stress Test Shifts in Factor Categories

We can stress test for shifts in factor categories and observe the impact on our portfolio. 

Here are the user inputs:
* `model_id`: The model we are using
* `portfolio_id`: Portfolio we are stress testing for.
* `factors_shifts_std`: Map of factor categories and their standard deviation scenario shift

In [ ]:
date = dt.date(2022, 5, 2)

model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

portfolio_id = "MPE2KR4BC4RKQ5KE"
pm = PortfolioManager(portfolio_id)

factors_shifts_std = {
   "Inflation": 0.5,
   "Real Rates": 3,
   "Economic Growth": -2
}
many_factors = model.get_many_factors()

std_dict = {factor: factors_shifts_std[factor.name]
            for factor in many_factors if factor.name in factors_shifts_std.keys()}
category_dict = {f.name: f.category for f in many_factors if f.category in factors_shifts_std.keys()}

# MacroScenario object
scenario = MacroScenario(date, std_dict)
exposure_df = pm.get_macro_exposure_table(macro_risk_model=model, date=scenario.date, factors=list(category_dict.keys()), group_by_factor_category=True)
portfolio_change = macro_stress_test(exposure_df, scenario, is_factor_category_stress=True).transpose().rename_axis("Factor Category")

styles = [dict(selector="caption", props=[("font-size", "120%"), ("font-weight", "bold"),("color", 'black')])]
portfolio_change = portfolio_change.style.set_caption("Stress Test: Shifts in Factor Categories and their Impact on Portfolio").format({"Percent Change": "{:.3f}%", "Portfolio Notional Change": "${:,.0f}", "Standard Deviation Shift": "{:.2f}"}, na_rep="N/A").background_gradient().set_table_styles(styles)

display(portfolio_change)

#### Stress Test Shifts in Factors

We can also stress test for shifts in individual factors and observe the impact on our portfolio. 

Here are the user inputs:
* `model_id`: The model we are using
* `portfolio_id`: Portfolio we are stress testing for.
* `factors_shifts_std`: Map of factors and their standard deviation scenario shifts

In [ ]:
date = dt.date(2022, 5, 2)

model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

portfolio_id = "MPE2KR4BC4RKQ5KE"
pm = PortfolioManager(portfolio_id)

factors_shifts_std = {
    "US 2Y Infl. Expec.": 0.5,
    "US 5Y Infl. Expec.": 0.5,
    "US 10Y Infl. Expec.": 0.5,
    "USD 10Y Real Rate": 3.0,
    "JPY 10Y Real Rate": 3.0,
    "EUR 10Y Real Rate": 3.0
}
many_factors = model.get_many_factors()
std_dict = {factor: factors_shifts_std[factor.name]
            for factor in many_factors if factor.name in factors_shifts_std.keys()}

# MacroScenario object
scenario = MacroScenario(date, std_dict)
exposure_df = pm.get_macro_exposure_table(macro_risk_model=model, date=scenario.date, factors=[factor.name for factor in scenario.factor_std_shifts.keys()])
portfolio_change = macro_stress_test(exposure_df, scenario, is_factor_category_stress=False).transpose().rename_axis("Factor")

styles = [dict(selector="caption", props=[("font-size", "130%"), ("font-weight", "bold"),("color", 'black')])]
portfolio_change = portfolio_change.style.set_caption("Stress Test: Shifts in Factors and their Impact on Portfolio").format({"Percent Change": "{:.3f}%", "Portfolio Notional Change": "${:,.0f}", "Standard Deviation Shift": "{:.2f}"}, na_rep="N/A").background_gradient().set_table_styles(styles)

# 1 factor std in absolute terms. 
factor_std_df = model.get_factor_standard_deviation(start_date=date, end_date=date, factors=factors_shifts_std.keys())
display(factor_std_df.style.set_caption(f"1 Standard Deviation of the Factors in Absolute Terms on {date}").format("{:.3f}").background_gradient().set_table_styles(styles))
display(portfolio_change)

## Which assets in the portfolio are in a Macro Regime?

In a macro regime, macro factors are the most significant in explaining the movement in asset price. To determine which
assets are in a macro regime, we look at the R Squared value, the proportion of the movement in asset price that is explained
by the macro factors. Assets with R Squared values above 65% are in a macro regime while values below 65% are driven
primarily by micro and idiosyncratic risk.

Once an asset is in a macro regime, we can look at the fair value gap to determine if it is rich or cheap.
The fair value gap metric is the difference between the spot price of an asset and the asset model value. An asset is
 undervalued when its fair value gap is negative while it is overvalued when it is a positive value.

Given a portfolio, we can find out how many assets are in a macro regime at a specific date. We can leverage the function
`get_fair_value_gap` and `get_r_squared` to determine the regime of each asset in the portfolio as well as its fair value
gap. Note that the fair value gap data is available both in absolute terms (in percentage) and in standard deviation terms.

Here are the user inputs:
* `portfolio_id`: Portfolio we are using. 
* `model_id`: The model we are using to get r_squared and fair value gap data.

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel, DataAssetsRequest as DataRequest, RiskModelUniverseIdentifierRequest, Unit
from gs_quant.markets.portfolio_manager import PortfolioManager
from IPython.display import display, display_html
import seaborn as sns
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

GsSession.use(Environment.PROD)

def annotate_plot(data, ax, index):
    for i in range(data.shape[0]):
        if data[index][i] == 'DHR UN':
            ax.text(data["R Squared"][i], data["Fair Value Gap"][i], s=data[index][i], size='x-large', color='r')
        else:
            ax.text(data["R Squared"][i], data["Fair Value Gap"][i], s=data[index][i], size='large')
        
# Time range
start_date = dt.date(2022, 5, 2)
end_date = dt.date(2022, 5, 2)

# Portfolio id and model
portfolio_id = "MPE2KR4BC4RKQ5KE"
model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

pm = PortfolioManager(portfolio_id)
positions = pm.get_latest_position_set().positions

universe = [position.identifier for position in positions]
universe_for_request = DataRequest(RiskModelUniverseIdentifierRequest.bbid, universe)
fvg_std_df = model.get_fair_value_gap(start_date=start_date, end_date=end_date, fair_value_gap_unit=Unit.STANDARD_DEVIATION, assets=universe_for_request)
r_squared_df = model.get_r_squared(start_date=start_date, end_date=end_date, assets=universe_for_request)

date = dt.date(2022, 5, 2).strftime("%Y-%m-%d")
fvg_one_date_df = fvg_std_df.loc[date, :].to_frame().rename(columns={date: "Fair Value Gap"})
r_squared_one_date_df = r_squared_df.loc[date, :].to_frame().rename(columns={date: "R Squared"})

fvg_rsq_df = r_squared_one_date_df.join(fvg_one_date_df).reset_index().round(2)

macro_rich = fvg_rsq_df[(fvg_rsq_df["R Squared"] > 65) & (fvg_rsq_df["Fair Value Gap"] > 0)].set_index("index").rename_axis("Asset").sort_values(by="Fair Value Gap", ascending=False)
macro_cheap = fvg_rsq_df[(fvg_rsq_df["R Squared"] > 65) & (fvg_rsq_df["Fair Value Gap"] < 0)].set_index("index").rename_axis("Asset").sort_values(by="Fair Value Gap", ascending=True)

styles = [dict(selector="caption", props=[("text-align", "right"), ("font-size", "130%"), ("font-weight", "bold"), ("color", 'black')])]

macro_rich_styler = macro_rich.style.set_table_attributes("style='display:inline'").set_caption("Macro Rich Assets in Portfolio").format({"R Squared": "{:.2f}%", "Fair Value Gap": "{:.2f}"}).background_gradient().set_table_styles(styles)
macro_cheap_styler = macro_cheap.style.set_table_attributes("style='display:inline'").set_caption("Macro Cheap Assets in Portfolio").format({"R Squared": "{:.2f}%","Fair Value Gap": "{:.2f}"}).background_gradient().set_table_styles(styles)

if fvg_rsq_df.shape[0] > 150:
    fvg_rsq_df = fvg_rsq_df.sample(n=150).reset_index(drop=True)

# Quadrant of assets that are macro cheap/rich and micro
fig, ax = plt.subplots(figsize=(35, 20))
sns.scatterplot(data=fvg_rsq_df, x="R Squared", y='Fair Value Gap', hue='index', legend=False)

ax.set_title(f"Macro Rich vs Macro Cheap Assets on {date}", fontsize='xx-large')
ax.set_xlabel("R Squared (Model Confidence) in %", fontsize='xx-large')
ax.set_ylabel("Fair Value Gap (sigma)", fontsize='xx-large')
ax.set_xlim(0, 100)
ax.set_ylim(-2.5, 2.5)

annotate_plot(fvg_rsq_df, ax, "index")

ax.text(x=75, y=2.2, s="Macro Rich", alpha=0.7, fontsize='xx-large',  color='g')
ax.text(x=25, y=2.2, s="Micro Regime", alpha=0.7, fontsize='xx-large', color='g')
ax.text(x=25, y=-2.2, s="Micro Regime", alpha=0.7, fontsize='xx-large', color='g')
ax.text(x=75, y=-2.2 , s="Macro Cheap", alpha=0.7, fontsize='xx-large', color='g')

ax.axhline(y=0, color='k', linestyle='dashed')
ax.axvline(x=65, color='k', linestyle='dashed')

# Plot Just macro cheap assets
fig, ax = plt.subplots(figsize=(25, 10))
sns.scatterplot(data=macro_cheap.reset_index(), x="R Squared", y="Fair Value Gap", hue="Asset", legend=False)
annotate_plot(macro_cheap.reset_index(), ax, "Asset")
ax.set_title(f"Macro Cheap Assets on {end_date}", fontsize='xx-large')
ax.set_ylabel("Fair Value Gap", fontsize="x-large")
ax.set_xlabel("Assets", fontsize="x-large")
ax.text(x=75, y=-2.0 , s="Macro Cheap", alpha=0.7, fontsize='xx-large', color='g')


plt.show()

The first plot above classifies assets into 3 groups: macro cheap, macro rich and assets in a micro regime. The second plot shows the bottom right of the first
plot, i.e all the assets that are macro cheap. 


### Asset Sensitivity to Factors

The asset Sensitivity to a macro factor is the percentage change in the asset price for 1 standard deviation move
in that macro factor. We can leverage the function `get_universe_sensitivity` of the `MacroRiskModel` class to get a
time series of daily asset sensitivity to macro factors for a list of assets.

Here are the parameters that are needed:
* `start_date` and `end_date`: Time range for which to get sensitivity for. 
* `model_id`: The model we are using.
* `asset`: Asset to get macro sensitivity for.

Below, we are querying a time series of daily sensitivity data over the time range for one of the macro cheap assets in the portfolio. 

#### Asset Sensitivity to Factor Categories

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel
from gs_quant.target.risk_models import RiskModelDataAssetsRequest as DataRequest, RiskModelUniverseIdentifierRequest
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def style_negative(v, props=''):
    return props if v < 0 else None


# Time range
start_date = dt.date(2022, 5, 2)
end_date = dt.date(2022, 5, 2)

# The model to get sensitivity data for
model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

# Universe
asset = "DHR UN"
universe = [asset]
universe_for_request = DataRequest(RiskModelUniverseIdentifierRequest.bbid, universe)

# Get asset sensitivity data
factor_sens_df = model.get_universe_sensitivity(start_date=start_date, end_date=end_date, assets=universe_for_request)
factor_data = model.get_factor_data(start_date=start_date, end_date=end_date).set_index('identifier')
factor_sens_df.columns = pd.MultiIndex.from_tuples([(factor_data.loc[x, 'factorCategory'], factor_data.loc[x, 'name']) for x in factor_sens_df.columns])
factor_sens_df = factor_sens_df.droplevel(level=1).sort_values(by=[asset], axis=1, ascending=False)
factor_categories = [f[0] for f in factor_sens_df.columns.values.tolist()]
factor_category_sens_df = pd.concat([factor_sens_df[category].agg(np.sum, axis=1).to_frame().rename(columns={0: category}) for category in factor_categories], axis=1)
factor_category_sens_df = factor_category_sens_df.loc[:, ~factor_category_sens_df.columns.duplicated(keep='first')]

asset_sens_df = factor_category_sens_df.transpose().sort_values(by=[asset], ascending=False).rename_axis("Macro Factor Category")

styles = [{'selector': "caption", 'props': [("font-size", "100%"), ("font-weight", "bold"), ("color", 'black')]},
          {'selector': 'thead','props': [("background-color", "silver"), ("font-size", "12px")]},
          {'selector': 'tr', 'props': [("background-color", "whitesmoke"), ("font-size", "12px")]},
          {'selector': 'td', 'props': [("background-color", "aliceblue"), ("font-size", "12px")]}]

asset_sens_styler = asset_sens_df.rename(columns={asset: f"Percent Change for 1std shift in macro factor category for {asset}"}).style.set_caption(f"Sensitivity of {asset} to macro factor Categories on {start_date}").format("{:.2f}%").applymap(style_negative, props='color:red;').set_table_styles(styles)
display(asset_sens_styler)

# Plot the portfolio percent change if every  factor category shifted 1 std
fig, ax = plt.subplots(figsize=(30, 10))
sns.barplot(data=asset_sens_df.reset_index().rename_axis("Macro Factor Category"), x="Macro Factor Category", y=asset, ax=ax)
ax.set_xlabel("Macro Factor Category", fontsize="x-large")
ax.set_ylabel("Percent Change", fontsize="x-large")
ax.set_title(f"Percent Change for 1 STD Shift in each Macro Factor Category for {asset} on {end_date}", fontsize="xx-large")
ax.tick_params(axis='x', labelrotation = 60)
plt.show()

#### Asset Sensitivity to Factors

Once we know an asset sensitivity to factor categories, we can also query its sensitivity to individual macro factors within a factor category.

Here are the user inputs:
* `model`: The model we are using.
* `factor_categories`: A list of factor categories whose factors we want sensitivity for. If empty, it will default to returning sensitivity to the top 2 positive and negative macro drivers
* `start_date` and `end_date`: Time range to get sensitivity data for an asset

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel
from gs_quant.target.risk_models import RiskModelDataAssetsRequest as DataRequest, RiskModelUniverseIdentifierRequest
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def style_negative(v, props=''):
    return props if v < 0 else None

# Time range
start_date = dt.date(2022, 5, 2)
end_date = dt.date(2022, 5, 2)

# The model to get data for
model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

# select factor_categories
factor_categories = ["Risk Aversion", "Real Rates", "Corporate Credit"]

# Universe
asset = "DHR UN"
universe = [asset]
universe_for_request = DataRequest(RiskModelUniverseIdentifierRequest.bbid, universe)

# Get asset sensitivity data
factor_sens_df = model.get_universe_sensitivity(start_date=start_date, end_date=end_date, assets=universe_for_request)
factor_data = model.get_factor_data(start_date=start_date, end_date=end_date).set_index('identifier')
factor_sens_df.columns = pd.MultiIndex.from_tuples([(factor_data.loc[x, 'factorCategory'], factor_data.loc[x, 'name']) for x in factor_sens_df.columns])
factor_sens_df = factor_sens_df.droplevel(level=1)
categories = [f[0] for f in factor_sens_df.columns.values.tolist()]

if factor_categories:
    factor_sens_df = factor_sens_df[factor_categories]
else:
    total_df = pd.concat([factor_sens_df[category].agg(np.sum, axis=1).to_frame().rename(columns={0: category}) for category in categories], axis=1).sort_values(by=[asset], axis=1, ascending=False)
    total_df = total_df.loc[:, ~total_df.columns.duplicated(keep='first')]
    top_drivers = total_df.columns.values.tolist()[0:2] + total_df.columns.values.tolist()[-2:]
    factor_sens_df = factor_sens_df[top_drivers]
    
factor_sens_df = factor_sens_df.transpose().rename_axis(["Factor Category", "Factor"])

# Style
styles = [{'selector': "caption", 'props': [("font-size", "100%"), ("font-weight", "bold"), ("color", 'black')]},
          {'selector': 'thead','props': [("background-color", "silver"), ("font-size", "12px")]},
          {'selector': 'tr', 'props': [("background-color", "gainsboro"), ("font-size", "12px"), ("border", "solid"), ("border-width", "1px")]},
          {'selector': 'td', 'props': [("background-color", "aliceblue"), ("font-size", "12px")]}]

asset_sens_styler = factor_sens_df.rename(columns={asset: f"Percent Change for 1std shift in macro factor for {asset}"}).style.set_caption(f"Sensitivity of {asset} to macro factors on {start_date}").format("{:.2f}%").applymap(style_negative, props='color:red;').set_table_styles(styles)
display(asset_sens_styler)

# Plot the portfolio percent change if every  factor category shifted 1 std
fig, ax = plt.subplots(figsize=(30, 10))
sns.barplot(data=factor_sens_df.sort_values(by=[asset], ascending=False).reset_index(), x="Factor", y=asset, ax=ax)
ax.set_xlabel("Macro Factor", fontsize="x-large")
ax.set_ylabel("Percent Change", fontsize="x-large")
ax.set_title(f"Percent Change for 1 STD Shift in each Macro Factor for {asset} on {end_date}", fontsize="xx-large")
ax.tick_params(axis='x', labelrotation = 60)
plt.show()

### Asset Sensitivity to Factors Over Time

We can query asset sensitivity to indiviual factors within a facotr category over time.

In [ ]:
from gs_quant.models.risk_model import MacroRiskModel
from gs_quant.target.risk_models import RiskModelDataAssetsRequest as DataRequest, RiskModelUniverseIdentifierRequest
import datetime as dt
import matplotlib.pyplot as plt

start_date = dt.date(2021, 1, 1)
end_date = dt.date(2022, 5, 2)

# The model to get data for
model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

macro_cheap_asset = "DHR UN"
factors = []
universe = [macro_cheap_asset]
universe_for_request = DataRequest(RiskModelUniverseIdentifierRequest.bbid, universe)

# Get asset sensitivity data
macro_factor_sensitivity = model.get_universe_sensitivity(start_date=start_date, end_date=end_date, assets=universe_for_request)
model_factors = model.get_factor_data(start_date=start_date, end_date=end_date).set_index('identifier')
macro_factor_sensitivity.columns = [model_factors.loc[x, 'name'] for x in macro_factor_sensitivity.columns]

drivers = macro_factor_sensitivity.loc[(macro_cheap_asset, end_date.strftime("%Y-%m-%d")), :].to_frame().droplevel(level=1, axis=1).sort_values(by=macro_cheap_asset, ascending=False)
drivers = drivers.index.values.tolist()
top_drivers = drivers[0:2] + drivers[-2:None]

# Let's get the universe sensitivity to macro factors over time
asset_sensitivity_across_time = macro_factor_sensitivity.loc[macro_cheap_asset, factors] if factors else macro_factor_sensitivity.loc[macro_cheap_asset, top_drivers]

fig, ax = plt.subplots(figsize=(20, 5))
asset_sensitivity_across_time.plot(ax=ax)
ax.set_title(f"Sensitivity of {universe[0]} to Top Macro Factors on {end_date} Over Time", fontsize='x-large')
ax.set_xlabel("Date")
ax.set_ylabel("% change in asset price for 1 std shift", fontsize='x-large')
plt.show()

### Regime Shift over Time

In a time range, we can pinpoint points in time when macro factors have primarily explained the variance in asset price over a period of time for some assets in our portfolio. 
In other words, we can observe when R Squared for an asset was above 65% and when it was below 65%. 

Here are the user inputs:
* `model_id`: The model we are using.
* `universe`: A list of assets to get historical R Squared data for.
* `start_date` and `end_date`: Time range to get R Squared data

In [ ]:
import datetime as dt
from gs_quant.target.risk_models import RiskModelDataAssetsRequest as DataRequest, RiskModelUniverseIdentifierRequest
from gs_quant.models.risk_model import MacroRiskModel
import matplotlib.pyplot as plt

start_date = dt.date(2021, 1, 1)
end_date = dt.date(2022, 5, 16)

model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

universe = ["DHR UN", "AMZN UN"]
universe_for_request = DataRequest(RiskModelUniverseIdentifierRequest.bbid, universe)

r_squared_df = model.get_r_squared(start_date=start_date, end_date=end_date, assets=universe_for_request)

fig, ax = plt.subplots(figsize=(15, 7))
r_squared_df.plot(ax=ax)
ax.set_title(f"Regime Shift from {start_date} to {end_date}", fontsize='x-large')
ax.set_ylabel("RSquared (in %)", fontsize='x-large')
ax.axhline(y=65, linestyle='dashed')
plt.show()

### Fair Value Gap Over Time

We can look at how fair value gap has changed historically.

In [ ]:


import datetime as dt
from gs_quant.target.risk_models import RiskModelDataAssetsRequest as DataRequest, RiskModelUniverseIdentifierRequest
from gs_quant.models.risk_model import MacroRiskModel
import matplotlib.pyplot as plt

start_date = dt.date(2022, 1, 1)
end_date = dt.date(2022, 5, 2)

model_id = "QI_US_EQUITY_LT"
model = MacroRiskModel.get(model_id)

universe = ["DHR UN"]
universe_for_request = DataRequest(RiskModelUniverseIdentifierRequest.bbid, universe)

# fvg returned in standard deviation terms by default
# To see fvg as percent, change the fair value gap unit to percent
fair_value_gap_std_df = model.get_fair_value_gap(start_date=start_date, end_date=end_date, fair_value_gap_unit=Unit.STANDARD_DEVIATION, assets=universe_for_request)

fig, ax = plt.subplots(figsize=(15, 7))
fair_value_gap_std_df.iloc[:, 0:4].plot(ax=ax)
ax.set_title(f"Fair Value Gap from {start_date} to {end_date}", fontsize='x-large')
ax.set_ylabel("Fair Value Gap (sigma)", fontsize='x-large')
ax.axhline(y=0, linestyle='dashed')
plt.show()


